In [8]:
import nest_asyncio
nest_asyncio.apply()
import aiohttp
import asyncio
import json
import pandas as pd
from datetime import datetime 
from datetime import timedelta


# UMS app information
ums_base_url = "https://edoofa-ums-90164.bubbleapps.io/version-test/api/1.1/obj"
ums_bearer_token = "786720e8eb68de7054d1149b56cc04f9"
ums_headers = {'Authorization': f'Bearer {ums_bearer_token}'}

# Asynchronous function to fetch data from a table
async def fetch_table_data(session, base_url, headers, table, constraints=None):
    records = []
    cursor = 0
    total_fetched = 0

    while True:
        params = {'limit': 100, 'cursor': cursor}
        if constraints:
            params['constraints'] = json.dumps(constraints)

        api_url = f"{base_url}/{table}"
        print(f"Fetching {table} data from {base_url}... Cursor: {cursor}")

        async with session.get(api_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch data from {table}: {await response.text()}")
                break

            data = await response.json()
            new_records = data['response']['results']
            records.extend(new_records)
            total_fetched += len(new_records)

            print(f"Fetched {len(new_records)} new records, Total fetched: {total_fetched}")

            cursor += 100

            if len(new_records) < 100:
                print(f"Exiting loop, fetched less than 100 records.")
                break

    df = pd.DataFrame(records)
    print(f"Fetched {len(df)} records for {table}.")
    return df
        
def get_percentage(number_of_present, total_sessions):
    if total_sessions == 0:
        return 0  # Avoid division by zero
    percent = (number_of_present / total_sessions) * 100
    #print(percent)
    return percent

async def post_processed_data(session, base_url, headers, processed_data):
    api_url = f"{base_url}/Attendance-Summary"
    
    for index, row in enumerate(processed_data):  
        data = row  
        
        async with session.post(api_url, headers=headers, json=data) as response:
            if response.status == 201:
                print(f"Successfully posted data for row {index}")
            else:
                print(f"Failed to post data for row {index}: {await response.text()}")

                

def update_attendance_summary(ums_student_df, ums_att_df, ums_att_summary_df, unique_dates):
    unique_students = ums_student_df['admissions-group-name'].unique()
    
    ums_att_df['date'] = pd.to_datetime(ums_att_df['date'])

    new_rows = []

    for student in unique_students:
        ewyl_group_name = ums_student_df[ums_student_df['admissions-group-name'] == student]['ewyl-group-name'].iloc[0]
        # Initialize the variable to store the last calculated average attendance
        last_calculated_avg_att = 0

        for date in unique_dates:
            # Check if a record already exists for this student and date
            if not ums_att_summary_df[
                (ums_att_summary_df['admissions-group-name'] == student) & 
                (ums_att_summary_df['first-day-of-month'] == date)
            ].empty:
                # If a record exists, skip to the next date
                continue

            # Fetch attendance data for the student up to the end of the previous month
            student_att_data_before_date = ums_att_df[
                (ums_att_df['admissions-group-name'] == student) &
                (ums_att_df['date'] < date)
            ]

            total_present = student_att_data_before_date['present'].sum()
            total_sessions = len(student_att_data_before_date)

            # Calculate the attendance percentage till the end of the previous month
            if total_sessions > 0:
                att_till_prev_month = get_percentage(total_present, total_sessions)
            else:
                # If there are no sessions before this date, use the last calculated value
                att_till_prev_month = last_calculated_avg_att

            # Update the last calculated average attendance for the next iteration
            last_calculated_avg_att = att_till_prev_month

            # Calculate month and year from first-day-of-month
            month = date.strftime('%B')
            year = date.year

            new_row = {
                'admissions-group-name': student,
                'ewyl-group-name': ewyl_group_name,
                'first-day-of-month': date.strftime('%Y-%m-%d'),
                'attendance-percentage': 0,
                'avg-att-percent-till-last-month': att_till_prev_month,
                'month': date.strftime('%B'),
                'year': date.year
            }
            new_rows.append(new_row)

    # Add new rows to the DataFrame
    if new_rows:
        ums_att_summary_df = pd.DataFrame(new_rows)

    return ums_att_summary_df

    
async def main():    
    async with aiohttp.ClientSession() as session:
        #fetch 
        ums_att_summary_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance-Summary")
        ums_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance")
        
        ums_student_df = await fetch_table_data(session, ums_base_url, ums_headers, "Student")       
        
        ums_att_summary_df['first-day-of-month'] = pd.to_datetime(ums_att_summary_df['first-day-of-month'])
        unique_dates = sorted(ums_att_summary_df['first-day-of-month'].unique())
                
        ums_att_summary_df = update_attendance_summary(ums_student_df, ums_att_df, ums_att_summary_df, unique_dates)
        ums_att_summary_df.to_csv("generated.csv", index=False)
        new_rows_list = ums_att_summary_df.to_dict(orient='records')
        
        await post_processed_data(session, ums_base_url, ums_headers, new_rows_list)   
        
await main()


Fetching Attendance-Summary data from https://edoofa-ums-90164.bubbleapps.io/version-test/api/1.1/obj... Cursor: 0
Fetched 42 new records, Total fetched: 42
Exiting loop, fetched less than 100 records.
Fetched 42 records for Attendance-Summary.
Fetching Attendance data from https://edoofa-ums-90164.bubbleapps.io/version-test/api/1.1/obj... Cursor: 0
Fetched 100 new records, Total fetched: 100
Fetching Attendance data from https://edoofa-ums-90164.bubbleapps.io/version-test/api/1.1/obj... Cursor: 100
Fetched 10 new records, Total fetched: 110
Exiting loop, fetched less than 100 records.
Fetched 110 records for Attendance.
Fetching Student data from https://edoofa-ums-90164.bubbleapps.io/version-test/api/1.1/obj... Cursor: 0
Fetched 11 new records, Total fetched: 11
Exiting loop, fetched less than 100 records.
Fetched 11 records for Student.
Successfully posted data for row 0
Successfully posted data for row 1
Successfully posted data for row 2
Successfully posted data for row 3
Successf